<a href="https://colab.research.google.com/github/Mariyselita/CIP/blob/main/Tarea7_Iris_GNB_kfold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ucimlrepo

In [ ]:
import pandas as pd
from ucimlrepo import fetch_ucirepo
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import KFold
from scipy.stats import norm

iris = fetch_ucirepo(id=53)

X = iris.data.features
y = iris.data.targets

In [ ]:
iris

{'data': {'ids': None,
  'features':      sepal length  sepal width  petal length  petal width
  0             5.1          3.5           1.4          0.2
  1             4.9          3.0           1.4          0.2
  2             4.7          3.2           1.3          0.2
  3             4.6          3.1           1.5          0.2
  4             5.0          3.6           1.4          0.2
  ..            ...          ...           ...          ...
  145           6.7          3.0           5.2          2.3
  146           6.3          2.5           5.0          1.9
  147           6.5          3.0           5.2          2.0
  148           6.2          3.4           5.4          2.3
  149           5.9          3.0           5.1          1.8
  
  [150 rows x 4 columns],
  'targets':               class
  0       Iris-setosa
  1       Iris-setosa
  2       Iris-setosa
  3       Iris-setosa
  4       Iris-setosa
  ..              ...
  145  Iris-virginica
  146  Iris-virginica
  147  

In [ ]:
class GaussianNBManual:
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.parameters = {}
        for cls in self.classes:
            X_cls = X[y == cls]
            self.parameters[cls] = {
                'mean': X_cls.mean(axis=0),
                'std': X_cls.std(axis=0),
                'prior': len(X_cls) / len(X)
            }

    def predict(self, X):
        y_pred = []
        for x in X:
            posteriors = []
            for cls in self.classes:
                prior = np.log(self.parameters[cls]['prior'])
                conditional = np.sum(np.log(self._pdf(cls, x)))
                posterior = prior + conditional
                posteriors.append(posterior)
            y_pred.append(self.classes[np.argmax(posteriors)])
        return np.array(y_pred)

    def _pdf(self, class_idx, x):
        mean = self.parameters[class_idx]['mean']
        std = self.parameters[class_idx]['std']
        return norm.pdf(x, mean, std).prod()

In [ ]:
iris = fetch_ucirepo(id=53)
X = iris.data.features.values
y = iris.data.targets['class'].values

In [ ]:
model = GaussianNBManual()
kf = KFold(n_splits=5, shuffle=True, random_state=42)
scores = []

In [ ]:
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    accuracy = np.mean(predictions == y_test)
    scores.append(accuracy)

<ipython-input-50-9fd0420e2e3a>:19: RuntimeWarning: divide by zero encountered in log
  conditional = np.sum(np.log(self._pdf(cls, x)))


In [ ]:
print("Cross-validated accuracies:", scores)
print("Mean accuracy:", np.mean(scores))

Cross-validated accuracies: [1.0, 0.9666666666666667, 0.9333333333333333, 0.9666666666666667, 0.9333333333333333]
Mean accuracy: 0.9600000000000002
